## Dataset Size vs Prediction Quality

**WIP**

In [1]:
from cvutils.rectlabel import io as rectlabel_io
import papermill as pm
import os.path as osp
import os

In [2]:
data_dir = osp.join(os.environ['CELLDOM_DATA_DIR'], 'data', 'dataset02')
full_image_list = rectlabel_io.list_dir(data_dir)
annot_image_list = [i for i in full_image_list if i.annot_exists]
len(annot_image_list)

160

In [4]:
import pandas as pd
pd.DataFrame(annot_image_list[:5])

,image_name,image_path,annot_path,annot_exists
0,BF_ST_001_APT_010Day2.jpg,/lab/data/celldom/data/dataset02/BF_ST_001_APT...,/lab/data/celldom/data/dataset02/annotations/B...,True
1,BF_ST_000_APT_004Day2.jpg,/lab/data/celldom/data/dataset02/BF_ST_000_APT...,/lab/data/celldom/data/dataset02/annotations/B...,True
2,BF_ST_001_APT_000Day1.jpg,/lab/data/celldom/data/dataset02/BF_ST_001_APT...,/lab/data/celldom/data/dataset02/annotations/B...,True
3,BF_ST_002_APT_002Day0.jpg,/lab/data/celldom/data/dataset02/BF_ST_002_APT...,/lab/data/celldom/data/dataset02/annotations/B...,True
4,BF_ST_002_APT_001Day3.jpg,/lab/data/celldom/data/dataset02/BF_ST_002_APT...,/lab/data/celldom/data/dataset02/annotations/B...,True


In [ ]:
run_name = 'dataset_size_vs_accuracy_n10'

parameters = dict(
    data_dir = osp.join(os.environ['CELLDOM_DATA_DIR'], 'data', 'dataset02')
    model_dir = osp.join('..', 'models', run_name),
    export_dir = osp.join('..', 'exports', run_name),
)
pm.execute_notebook(
    osp.join('templates', 'training_quantification.ipynb'),
    osp.join('renders', run_name + '.ipynb')
)